In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [22]:
# Define transformation for MNIST (e.g., convert to tensor)
transform = transforms.Compose([transforms.ToTensor()
                               ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the full MNIST training dataset
mnist_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Create a DataLoader for the subset
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=True)


# Select the first 600 images for training
subset_indices_1 = list(range(600))
mnist_subset_1 = Subset(mnist_train, subset_indices_1)

# Create a DataLoader for the subset
# train_loader_1 = DataLoader(mnist_subset_1, batch_size=64, shuffle=True)

# Select the first 600 images for training
subset_indices_2 = list(range(600,1200))
mnist_subset_2 = Subset(mnist_train, subset_indices_2)

# Create a DataLoader for the subset
# train_loader_2 = DataLoader(mnist_subset_2, batch_size=64, shuffle=True)

def get_data_loader(train_data, batch_size):
    return DataLoader(train_data, batch_size=batch_size, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)  # 3 input channels (for CIFAR-10), 10 output channels
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  # 10 input channels, 20 output channels
        self.fc1 = nn.Linear(500, 50)  # We'll compute 500 dynamically based on input size
        self.fc2 = nn.Linear(50, 10)  # 10 output features (CIFAR-10 has 10 classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))  # RELU activation, max pooling
        x = F.relu(F.max_pool2d(self.conv2(x), 2))  # RELU activation, max pooling
        x = x.view(x.size(0), -1)  # Dynamically flatten the output
        x = F.relu(self.fc1(x))  # RELU activation
        x = self.fc2(x)
        return x


In [9]:
import torch.optim as optim

def evaluate(model,loader) -> float:
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def train(model,train_loader,batch_size=64,EPOCHS=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        running_correct = 0.0
        model = model.to(device)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (labels == predicted).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = running_correct / (len(train_loader) * batch_size)
        test_accuracy = evaluate(model,test_loader)
        model.train()
        print(f"Epoch [{epoch+1}/{EPOCHS}]: Loss: {epoch_loss}, Train accuracy: {epoch_accuracy}, Test accuracy: {test_accuracy}")
    return  model

In [10]:
import torch
import copy

# Assume model_list is a list of models
def average_model_weights(model_list):
    avg_model_state_dict = copy.deepcopy(model_list[0])
    
    for key in avg_model_state_dict:
        avg_model_state_dict[key] = torch.zeros_like(avg_model_state_dict[key])

    num_models = len(model_list)
    for model_state_dict in model_list:
        for key in avg_model_state_dict:
            avg_model_state_dict[key] += model_state_dict[key]
    
    for key in avg_model_state_dict:
        avg_model_state_dict[key] /= num_models

    return avg_model_state_dict
    


In [29]:
def federated_learning(batch_size=50, init=True):
    train_loaders = [get_data_loader(mnist_subset_1, batch_size), get_data_loader(mnist_subset_2, batch_size)]

    def server_execute(t,E,K, init=True):
        if init:
            w0 = copy.deepcopy((SimpleCNN()).state_dict())
        else:
            w0 = None
        for i in range(t):
            weights = []
            for k in range(K):
                weights.append(ClientUpdate(E,k,w0))
            w0 = average_model_weights(weights)
            avg_model = SimpleCNN()
            avg_model.load_state_dict(w0)
            print(f"round {i} accuracy: {evaluate(avg_model,test_loader)}")
        return avg_model
        
    def ClientUpdate(E,k, w=None):
        model = SimpleCNN()
        if w is not None:
            model.load_state_dict(state_dict=w)
        model = train(model,train_loaders[k],batch_size=batch_size, EPOCHS=E)
        print(f"k={k}: {evaluate(model,test_loader)}")
        return model.state_dict()
    
    return server_execute(10,5,2, init=init)

Train model without Same init

In [23]:
modelInitF = federated_learning(init=False)

Epoch [1/5]: Loss: 2.301084021727244, Train accuracy: 0.0625, Test accuracy: 0.1165
Epoch [2/5]: Loss: 2.287939806779226, Train accuracy: 0.10286458333333333, Test accuracy: 0.1068
Epoch [3/5]: Loss: 2.2669325868288674, Train accuracy: 0.10807291666666667, Test accuracy: 0.1312
Epoch [4/5]: Loss: 2.2315643628438315, Train accuracy: 0.12630208333333334, Test accuracy: 0.1474
Epoch [5/5]: Loss: 2.1639700333277383, Train accuracy: 0.15625, Test accuracy: 0.1724
k=0: 0.1724
Epoch [1/5]: Loss: 2.303830146789551, Train accuracy: 0.08333333333333333, Test accuracy: 0.0999
Epoch [2/5]: Loss: 2.2971599102020264, Train accuracy: 0.08723958333333333, Test accuracy: 0.1174
Epoch [3/5]: Loss: 2.287758469581604, Train accuracy: 0.11458333333333333, Test accuracy: 0.1466
Epoch [4/5]: Loss: 2.2671692967414856, Train accuracy: 0.140625, Test accuracy: 0.1668
Epoch [5/5]: Loss: 2.2356728116671243, Train accuracy: 0.13671875, Test accuracy: 0.193
k=1: 0.193
round 0 accuracy: 0.1809
Epoch [1/5]: Loss: 2.2

Train Models with same init

In [25]:
modelInitT = federated_learning(init=True)

Epoch [1/5]: Loss: 2.3065717617670694, Train accuracy: 0.08333333333333333, Test accuracy: 0.1115
Epoch [2/5]: Loss: 2.2989671428998313, Train accuracy: 0.08723958333333333, Test accuracy: 0.1039
Epoch [3/5]: Loss: 2.2886388103167215, Train accuracy: 0.09114583333333333, Test accuracy: 0.1014
Epoch [4/5]: Loss: 2.2759340604146323, Train accuracy: 0.10026041666666667, Test accuracy: 0.1001
Epoch [5/5]: Loss: 2.2518537839253745, Train accuracy: 0.10026041666666667, Test accuracy: 0.1001
k=0: 0.1001
Epoch [1/5]: Loss: 2.3050908843676248, Train accuracy: 0.08333333333333333, Test accuracy: 0.111
Epoch [2/5]: Loss: 2.2990620930989585, Train accuracy: 0.09765625, Test accuracy: 0.1027
Epoch [3/5]: Loss: 2.291678865750631, Train accuracy: 0.09765625, Test accuracy: 0.1025
Epoch [4/5]: Loss: 2.282223383585612, Train accuracy: 0.09375, Test accuracy: 0.1013
Epoch [5/5]: Loss: 2.2647220889727273, Train accuracy: 0.09505208333333333, Test accuracy: 0.1028
k=1: 0.1028
round 0 accuracy: 0.1271
Epoc

In [26]:
# compare the two models

print(f"Model with different initialisation: {evaluate(modelInitF,test_loader)}")

print(f"Model with same initialisation: {evaluate(modelInitT,test_loader)}")

Model with different initialisation: 0.3845
Model with same initialisation: 0.3886


Trying different batch_size

In [27]:
batch_sizes = [10, 20, 30]
models = []

for batch_size in batch_sizes:
    models.append(federated_learning(batch_size, init=True))

Epoch [1/5]: Loss: 2.2928849856058755, Train accuracy: 0.018229166666666668, Test accuracy: 0.1073
Epoch [2/5]: Loss: 2.157798461119334, Train accuracy: 0.03046875, Test accuracy: 0.2123
Epoch [3/5]: Loss: 2.0744644343852996, Train accuracy: 0.034895833333333334, Test accuracy: 0.1802
Epoch [4/5]: Loss: 1.9658590177694957, Train accuracy: 0.04192708333333333, Test accuracy: 0.2484
Epoch [5/5]: Loss: 1.9013889710108438, Train accuracy: 0.04609375, Test accuracy: 0.2758
k=0: 0.2758
Epoch [1/5]: Loss: 2.299170172214508, Train accuracy: 0.019270833333333334, Test accuracy: 0.1964
Epoch [2/5]: Loss: 2.2006404797236123, Train accuracy: 0.03125, Test accuracy: 0.2216
Epoch [3/5]: Loss: 2.069770489136378, Train accuracy: 0.04010416666666667, Test accuracy: 0.2019
Epoch [4/5]: Loss: 2.015862359603246, Train accuracy: 0.04114583333333333, Test accuracy: 0.2386
Epoch [5/5]: Loss: 1.9278668145338693, Train accuracy: 0.04765625, Test accuracy: 0.2463
k=1: 0.2463
round 0 accuracy: 0.2717
Epoch [1/5]

In [28]:
for model, batch_size in zip(models, batch_sizes):
    print(f"Batch size: {batch_size}, Accuracy: {evaluate(model, test_loader)}")

Batch size: 10, Accuracy: 0.3365
Batch size: 20, Accuracy: 0.3955
Batch size: 30, Accuracy: 0.3837
